In [8]:
import pandas as pd
import os

# Definir la ruta a la carpeta ODN
odd_folder = r'C:\Users\trico\Desktop\Internas 2024\ODD'

# Crear una lista para almacenar los datos de todos los archivos
all_data = []

# Recorrer todos los archivos en la carpeta ODD
for file_name in os.listdir(odd_folder):
    # Verificar si el archivo es un Excel moderno (xlsx)
    if file_name.endswith('.xlsx'):
        # Definir la ruta completa del archivo
        file_path = os.path.join(odd_folder, file_name)
        try:
            # Leer el archivo Excel, con el encabezado en la fila 9
            df = pd.read_excel(file_path, header=8, usecols='A:H')
            # Agregar los datos a la lista
            all_data.append(df)
        except Exception as e:
            print(f"Error leyendo el archivo {file_name}: {e}")

# Combinar todos los datos en un solo DataFrame
if all_data:
    combined_data = pd.concat(all_data, ignore_index=True)

    # Filtrar los datos donde la columna DEPTO es Montevideo
    montevideo_odd_data = combined_data[combined_data['DEPTO'] == 'Montevideo']

    # Mostrar los primeros registros del DataFrame filtrado
    print(montevideo_odd_data.head())

    # Guardar el DataFrame filtrado en un archivo CSV
    montevideo_odd_data.to_csv('montevideo_odd_dataset.csv', index=False)
else:
    print("No se encontraron datos válidos en los archivos Excel.")


        ACTO               CONVOCATORIA       DEPTO CIRCUITO SERIES  \
0 2024-06-30  Asamblea_Popular_-_O.D.D.  Montevideo        1    AAA   
1 2024-06-30  Asamblea_Popular_-_O.D.D.  Montevideo        6    AAA   
2 2024-06-30  Asamblea_Popular_-_O.D.D.  Montevideo        7    AAA   
3 2024-06-30  Asamblea_Popular_-_O.D.D.  Montevideo       11    AAA   
4 2024-06-30  Asamblea_Popular_-_O.D.D.  Montevideo       13    AAA   

      ESCRUTINIO  HOJA CNT_VOTOS  
0  Departamental   969         1  
1  Departamental  9600         1  
2  Departamental  1326         1  
3  Departamental  1326         1  
4  Departamental  1326         2  


C:\Users\trico\AppData\Local\Temp\ipykernel_15336\2324535861.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat(all_data, ignore_index=True)


In [9]:
# Renombrar la columna 'CONVOCATORIA' a 'PARTIDO'
montevideo_odd_data.rename(columns={'CONVOCATORIA': 'PARTIDO'}, inplace=True)

# Función para modificar los valores de la columna 'PARTIDO'
def modificar_partido_odd(valor):
    # Separar el valor por '_-_' y tomar la primera parte
    partido = valor.split('_-_')[0]
    # Reemplazar '_' por espacios y agregar 'Partido' al inicio
    partido = partido.replace('_', ' ')
    # Eliminar el sufijo ' O.D.N.' si existe
    if 'O.D.D.' in partido:
        partido = partido.replace(' O.D.D.', '')
    return partido

# Aplicar la función a la columna 'PARTIDO'
montevideo_odd_data['PARTIDO'] = montevideo_odd_data['PARTIDO'].apply(modificar_partido_odd)

# Verificar si la columna 'ACTO' existe y eliminarla si es el caso
if 'ACTO' in montevideo_odd_data.columns:
    montevideo_odd_data.drop(columns=['ACTO'], inplace=True)

# Mostrar los primeros registros del DataFrame modificado
montevideo_odd_data.head()


C:\Users\trico\AppData\Local\Temp\ipykernel_15336\1171292560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montevideo_odd_data.rename(columns={'CONVOCATORIA': 'PARTIDO'}, inplace=True)
C:\Users\trico\AppData\Local\Temp\ipykernel_15336\1171292560.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montevideo_odd_data['PARTIDO'] = montevideo_odd_data['PARTIDO'].apply(modificar_partido_odd)
C:\Users\trico\AppData\Local\Temp\ipykernel_15336\1171292560.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

,PARTIDO,DEPTO,CIRCUITO,SERIES,ESCRUTINIO,HOJA,CNT_VOTOS
0,Asamblea Popular,Montevideo,1,AAA,Departamental,969,1
1,Asamblea Popular,Montevideo,6,AAA,Departamental,9600,1
2,Asamblea Popular,Montevideo,7,AAA,Departamental,1326,1
3,Asamblea Popular,Montevideo,11,AAA,Departamental,1326,1
4,Asamblea Popular,Montevideo,13,AAA,Departamental,1326,2


In [10]:
montevideo_odd_data.to_csv('montevideo_odd_dataset_modificado.csv', index=False)

In [11]:
import pandas as pd

# Leer el archivo CSV de referencia
ref_df = pd.read_csv('votos_por_barrio_pn_mapeado_odd.csv', delimiter=';')

# Asegurarse de que la columna 'SERIE' sea de tipo string
ref_df['SERIE'] = ref_df['SERIE'].astype(str)

# Crear un diccionario para mapear series a zonas
series_to_zona = {}
for index, row in ref_df.iterrows():
    series = row['SERIE'].split()  # Dividir las series si están separadas por espacios
    for serie in series:
        series_to_zona[serie] = row['ZONA']

# Función para mapear serie a zona
def obtener_zona(serie):
    series = str(serie).split()  # Asegurarse de que la serie sea una cadena y dividirla
    zonas = [series_to_zona.get(s, 'Desconocido') for s in series]  # Obtener la zona para cada serie
    return ' '.join(set(zonas))  # Devolver las zonas únicas

# Aplicar la función al DataFrame existente (montevideo_odd_data)
montevideo_odd_data['ZONA'] = montevideo_odd_data['SERIES'].apply(obtener_zona)

# Mostrar los primeros registros del DataFrame modificado
print(montevideo_odd_data.head())

# Guardar el DataFrame modificado en un archivo CSV
montevideo_odd_data.to_csv('montevideo_odd_dataset_con_zona.csv', index=False)


            PARTIDO       DEPTO CIRCUITO SERIES     ESCRUTINIO  HOJA  \
0  Asamblea Popular  Montevideo        1    AAA  Departamental   969   
1  Asamblea Popular  Montevideo        6    AAA  Departamental  9600   
2  Asamblea Popular  Montevideo        7    AAA  Departamental  1326   
3  Asamblea Popular  Montevideo       11    AAA  Departamental  1326   
4  Asamblea Popular  Montevideo       13    AAA  Departamental  1326   

  CNT_VOTOS          ZONA  
0         1  Ciudad Vieja  
1         1  Ciudad Vieja  
2         1  Ciudad Vieja  
3         1  Ciudad Vieja  
4         2  Ciudad Vieja  


C:\Users\trico\AppData\Local\Temp\ipykernel_15336\1701394476.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montevideo_odd_data['ZONA'] = montevideo_odd_data['SERIES'].apply(obtener_zona)
